# Search and Path Planning

**Note:** *This project uses and extends portions of the* UC Berkeley Pacman Project 1: Search in Pacman, *as well as portions of the following lecture resources:*
- [Stanford Autumn 2019 CS221 course Artificial Intelligence: Principles and Techniques](https://www.youtube.com/watch?v=J8Eh7RqggsU&list=PLoROMvodv4rO1NB9TD4iUZ3qghGEGtqNX&index=1)
- [RRT's](https://www.cs.cmu.edu/~motionplanning/lecture/lec20.pdf) from the CMU Fall 2010 Robotic Motion Planning course

In [ ]:
# Licensing Information:  You are free to use or extend these projects for 
# educational purposes provided that (1) you do not distribute or publish 
# solutions, (2) you retain this notice, and (3) you provide clear 
# attribution to UC Berkeley, including a link to http://ai.berkeley.edu.
# 
# Attribution Information: The Pacman AI projects were developed at UC Berkeley.
# The core projects and autograders were primarily created by John DeNero
# (denero@cs.berkeley.edu) and Dan Klein (klein@cs.berkeley.edu).
# Student side autograding was added by Brad Miller, Nick Hay, and
# Pieter Abbeel (pabbeel@cs.berkeley.edu).

## Part 1: Introduction to Search and AI Definitions

In this project, we'll be talking about algorithms for search or path-planning.

What are some examples of problems that might involve searching for a solution? Typically with search or path-planning problems, the first thing that comes to your mind might be searching through a maze, or trying to find an exit after getting lost in IKEA. However, as we'll see in a little bit, you can apply these algorithms to a wide variety of problems for reasoning and making decisions.

![riddle](images/sheep_wolf_cabbage.webp)

Suppose we have the following puzzle: a farmer needs to transport a wolf, a sheep, and a cabbage across a river. The farmer has a boat that can only carry one item at a time, and the wolf cannot be left alone with the sheep (since the wolf will eat the sheep), and sheep cannot be left alone with the cabbage (since it'll eat the cabbage). How can the farmer transport all three across the river? Take some time to think about this puzzle. How many moves can you solve it in?

![xkcd_logic_boat](images/logic_boat.png)

How did you approach solving this problem? What you might've done is considered all the possible moves that the farmer can make while preventing putting the sheep and wolf together, and the sheep and cabbage together. You might've also figured out that you had to go back and forth across the river multiple times and might've backtracked your solution after figuring out that a specific set of actions may not work. Although this may have not been what originally came to your mind, this is also an example of a **search** or **path-planning** problem, where we want to find a sequence of actions that can take us from some initial starting condition to the goal condition.

One thing to note about this specific puzzle search problem (and many other search problems as well) is that different moves that you make will have *different consequences* in the future. For example, you can never move the wolf first from the starting position or the cabbage first from the starting position. Therefore, finding solutions to a search problem requires taking into account potential consequences of your actions in the future.

![rubicks_cube](images/rubicks_cube.jpg)

Another example of a search problem is solving a Rubik's cube. Just like how we needed to find the sequence of actions that could bring the farmer, wolf, sheep, and cabbage to the other side of the river, we need to find the sequence of actions that can bring the Rubik's cube from a scrambled state to a solved state.

Before we dive into some algorithms for solving these types of problems, let's first formalize our search problem for our sheep, wolf, and cabbage example. We will be using these definitions in the scope of what we call a **state-based model**.

A **state** is a specific configuration of the environment. The start state of our puzzle above would be the farmer, wolf, sheep, and cabbage on the left side of the river, and the goal state would be the farmer, wolf, sheep, and cabbage on the right side of the river. Another example of a random state would be the farmer, wolf, and cabbage on the left side of the river, and the sheep on the right side of the river.

![state_example](images/puzzle_sheep_right.webp)

The **state space** is the set of ALL possible states that the environment can be in. This would include the start state, the goal state, and all the other states that can be reached from the start state. For our puzzle, the state space would be all the possible combinations of the farmer, wolf, sheep, and cabbage on either side of the river.

An **action** is a move that can be made to go from one state to another. To reach the state above with the sheep on the right side of the river, the action from the start state would be moving the sheep to the right side of the river. Afterwards, another action (although not so useful one) could be moving the sheep back to the left side of the river.

![action_example](images/puzzle_sheep_boat.webp)

The **transition function** is a function that takes in a state and an action, and returns the next state. In math notation, you might see a transition function defined as:

$$T(x_i, a) = x_{i+1}$$

where $x_i$ is the current state, $a$ is the action, and $x_{i+1}$ is the next state.

## Part 2: Planning with Graphs

If you are doing this project on your own computer, make sure that you have `Anaconda` or `Miniconda` installed. If you don't, you can download and install it from [here](https://docs.anaconda.com/free/anaconda/install/index.html).

Afterwards, you can create a new environment with the following command:

```bash
conda create -n planning_env python=3.8 -y
```

Then, activate the environment:

In [12]:
!conda activate planning_env

usage: conda [-h] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'env')


When you are taking a break or finished working on the project, you can deactivate the environment:

```bash
conda deactivate
```

![pacman_maze](images/maze.png)

Let's consider a Pacman world, where Pacman can move in four directions: up, down, left, and right. The world is represented as a grid, where each cell is either empty or contains a wall. Pacman can move to any empty cell, but cannot move to a cell with a wall. Since Pacman is an autonomous entity in our AI search problem, we will be referring to Pacman as the **agent**.

If your were the agent (Pacman), how would you try to find a path from the start state (the bottom right corner) to the goal state (the bottom left corner)? Take a second to attempt the maze yourself using the arrow or WASD keys after running the following cell and thinking about the process you took to get there.

In [11]:
import os

%cd planning_project_solution
!python pacman.py -z 0.5 -l bigMaze

[Errno 2] No such file or directory: 'planning_project_solution'
/home/jerredchen/code/smartcore/ai-technologies-teacher/level1/planning_project_solution


## Part 3: Planning with Sampling

So now that we've learned that A* is guaranteed to find an optimal path, we're pretty much all done right? Well, not quite. These algorithms are great for search in a grid world like we were doing before, but now let's suppose we're creating a robot that needs to plan inside a house or somewhere with a lot of empty space.

![dijkstra_robot_planning](images/Dijkstras_progress_animation.gif)

We could *maybe* still use these graph-planning algorithms, but we'd have to discretize the state space into a grid, and that could be pretty computationally expensive to 1. hold all that information in memory, and 2. have to search through all those nodes of empty space.

![robot_arm](images/kuka_iiwa_arm.png)

Even worse, let's consider a robot arm with 7 degrees of freedom, like seen above. In order to search through all the possible configurations of the robot arm, we'd have to search through a 7-dimensional configuration space! Therefore, we can see that the memory and computational requirements grow exponentially as the dimensionality of the state space increases, which is really bad.

So what can we do? A* is a really nice algorithm that ensures completion and optimality, but it isn't really feasible to apply for large, higher dimensional spaces. Instead, we can sacrifice completion and optimality to develop an algorithm that is much more efficient and practical. **Sampling** is the act of randomly selecting a point from a given distribution or a dataset. A lot of the times, computer science researchers will develop algorithms that utilize sampling since they can sometimes be a lot more efficient and faster than deterministic, or non-random algorithms. For Pacman moving through a lot of empty space, we'll be looking at a sampling-based planning algorithm called **Rapidly-exploring Random Trees (RRT)**.


Sources for images:

- [Professor Layton](https://en.wikipedia.org/wiki/Professor_Layton)
- [XKCD](https://xkcd.com/1134/)
- [Dijkstra's Algorithm on Wikipedia](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)